# Video concatenation v1
__Author:__ Jack Friedman <br>
__Purpose:__ Creates functions to generate clip intros and podlight intro 
__Updates since last version:__ Added LLM to make clip intros smoother 

In [3]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/local/bin/ffmpeg"  # Update this path as needed

from moviepy.editor import VideoFileClip, ImageClip, AudioClip, AudioFileClip, concatenate_videoclips
import scipy
import re

from transformers import AutoProcessor, BarkModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import HuggingFaceHub

In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'YOUR_TOKEN_HERE'

## Instantiate model

In [20]:
processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

voice_preset = "v2/en_speaker_6"

## Function 1: Make clip intro

### 1A) Text prompt LLM refinement 

In [18]:
# Helper function that extracts output from mistral code
def extract_output(text):
    pattern = r"\[/INST\]\s*\n\s*(.*)"

    match = re.search(pattern, text)

    if match:
        user_prompt = match.group(1).strip()
        output = user_prompt
    else:
        output = "Hi! Welcome to your custom podlight. Enjoy!"
    return output

def create_clip_intro_prompt(podcast, chapter_description):
    template = """<s>[INST] Clean up the prompt sentence, adding/removing transition words around the chapter description to make the sentence clearer. Only respond with the output sentence. The prompts should all have the template "Coming up, we've got a clip about [chapter-description] from [podcast-name]. Enjoy!"

    For example, if the chapter description is "Big Tech's involvement in political process' from the 'a16z' podcast, you would respond with 'Coming up, we've got a clip about big tech's involvement in the political process from a16z. Enjoy!"

    For example, if the chapter description is "Thoughts on recent TikTok legislation' from the 'All-In' podcast, you would respond with 'Coming up, we've got a clip about the recent Tiktok legislation from All-In. Enjoy!"

    For example, if the chapter description is "Q: Would either of you ever consider running for office?' from the 'VC20' podcast, you would respond with 'Coming up, we've got a clip about running for office from All-In. Enjoy!"

    Chapter description: {chapter_description}
    Podcast: {podcast}
    [/INST]
    """
    prompt = ChatPromptTemplate.from_template(template)

    # TRAINING LOOP
    model = HuggingFaceHub(
        repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
        task="text-generation",
        model_kwargs={
            "max_new_tokens": 30,
            "top_k": 30,
            "temperature": 0.1,
            "repetition_penalty": 1.03,
        },
    )

    chain = (
        {"chapter_description": RunnablePassthrough(), "podcast": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    output = chain.invoke({"chapter_description": chapter_description, "podcast":podcast})
    output = extract_output(output)
    return output


output = create_clip_intro_prompt('Huberman Lab', "Product Not Working")
print(output)

Coming up, we've got a clip about a product not working from Huberman Lab. Enjoy!


In [22]:
# Creates the intro segment for a video clip given the title of the podcast and the subtitle of the chapter
def make_clip_intro(podcast, chapter_description, model, processor, voice_preset, logo_filepath):
    text_prompt = create_clip_intro_prompt(podcast, chapter_description)
    inputs = processor(text_prompt, voice_preset=voice_preset)

    # Run voice generator
    audio_array = model.generate(**inputs)
    audio_array = audio_array.cpu().numpy().squeeze()

    # Create audio clip
    sample_rate = model.generation_config.sample_rate
    scipy.io.wavfile.write("audio_placeholder.wav", rate=sample_rate, data=audio_array)
    audio_clip = AudioFileClip("audio_placeholder.wav")  # creating an AudioCLipo from the audio array was super finicky so this was a workaround

     # Create an ImageClip object with the duration set to the audio clip's duration
    image_clip = ImageClip(logo_filepath, duration=audio_clip.duration)

    # Create output video
    output_clip = image_clip.set_audio(audio_clip)
    output_filepath = podcast + '_' + chapter_description + '_intro.mp4'
    output_clip.write_videofile(output_filepath, fps=1) 
    return output_clip
    

In [23]:
podcast = "a16z"
chapter_description = "Big Tech's involvement in political process"
VOICE_PRESET = "v2/en_speaker_6"
video_clip = make_clip_intro(podcast, chapter_description, model, processor, VOICE_PRESET, "logo.png")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used.

Moviepy - Building video a16z_Big Tech's involvement in political process_intro.mp4.
MoviePy - Writing audio in a16z_Big Tech's involvement in political process_introTEMP_MPY_wvf_snd.mp3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


MoviePy - Done.
Moviepy - Writing video a16z_Big Tech's involvement in political process_intro.mp4



Moviepy - Done !
Moviepy - video ready a16z_Big Tech's involvement in political process_intro.mp4


## Function 2: Make show intro

In [24]:
def create_show_intro_prompt(user_prompt):
    template = """<s>[INST]Turn the user's prompt into a one sentence intro using the template: 'Hi! We've created a podlight for you all about [user's topic]. Enjoy!'

    For example, if the user asks for 'podcast about AI', you would respond with 'Hi! We've created a podlight for you all about AI. Enjoy!'

    User prompt: {user_prompt}[/INST]
    """
    prompt = ChatPromptTemplate.from_template(template)

    # TRAINING LOOP
    model = HuggingFaceHub(
        repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
        task="text-generation",
        model_kwargs={
            "max_new_tokens": 30,
            "top_k": 30,
            "temperature": 0.1,
            "repetition_penalty": 1.03,
        },
    )

    chain = (
        {"user_prompt": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    output = chain.invoke(user_prompt)
    output = extract_output(output)
    return output


prompt = "Create me a podcast about Google"
output = create_show_intro_prompt(prompt)
print(output)

Hi! We've created a podlight for you all about Google. Enjoy!


In [46]:
# Creates the intro segment for a video clip given the title of the podcast and the subtitle of the chapter
def make_whole_show_intro(user_prompt, model, processor, voice_preset, logo_filepath):
    intro_prompt = create_show_intro_prompt(user_prompt)
    inputs = processor(intro_prompt, voice_preset=voice_preset)

    # Run voice generator
    audio_array = model.generate(**inputs)
    audio_array = audio_array.cpu().numpy().squeeze()

    # Get audio clip
    sample_rate = model.generation_config.sample_rate
    scipy.io.wavfile.write("audio_placeholder.wav", rate=sample_rate, data=audio_array)
    audio_clip = AudioFileClip("audio_placeholder.wav")  # creating an AudioCLipo from the audio array was super finicky so this was a workaround

     # Create an ImageClip object with the duration set to the audio clip's duration
    image_clip = ImageClip(logo_filepath, duration=audio_clip.duration)

    # Create output video
    output_clip = image_clip.set_audio(audio_clip)
    output_filepath = 'podlight_intro_clip.mp4'
    output_clip.write_videofile(output_filepath, fps=1) 
    return output_clip

In [47]:
VOICE_PRESET = "v2/en_speaker_6"
user_prompt = 'Make me a podcast about why Carter should buy a monitor'
make_whole_show_intro(user_prompt, model, processor, VOICE_PRESET, 'logo.png')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used.

Moviepy - Building video podlight_intro_clip.mp4.
MoviePy - Writing audio in podlight_intro_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video podlight_intro_clip.mp4



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Moviepy - Done !
Moviepy - video ready podlight_intro_clip.mp4


In [ ]:
clip1 = VideoFileClip("videos/waving.MOV")
clip2 = VideoFileClip("videos/fellas.MOV")
final_clip = concatenate_videoclips([clip1,clip2])
final_clip.write_videofile("concat_v0.mp4")